In [3]:
from problog.program import PrologString
from problog.logic import Term, Var, Constant, Clause, AnnotatedDisjunction, And, Or, Not, AggTerm
import inspect

class ProblogParser:
    """
    解析Problog代码并生成带有类型标注的结构化列表
    """
    
    def __init__(self):
        self.type_mapping = {
            Term: "term",
            Var: "variable",
            Constant: "constant",
            Clause: "clause",
            AnnotatedDisjunction: "annotated_disjunction",
            And: "conjunction",
            Or: "disjunction",
            Not: "negation",
            AggTerm: "aggregation"
        }
    
    def parse(self, code_string):
        """
        解析ProbLog代码并返回带有类型标注的结构化列表
        
        :param code_string: ProbLog代码字符串
        :return: 带有类型标注的结构化列表
        """
        # 使用PrologString解析代码
        program = PrologString(code_string)
        
        # 处理解析结果
        result = []
        for item in program:
            result.append(self._process_item(item))
            
        return result
    
    def _process_item(self, item):
        """
        处理单个解析项并添加类型标注
        
        :param item: 解析项
        :return: 带有类型标注的字典
        """
        # 确定项的类型
        item_type = type(item)
        type_name = self.type_mapping.get(item_type, "unknown")
        
        # 创建基本结构
        result = {
            "type": type_name,
            "content": str(item),
            "raw": item
        }
        
        # 根据不同类型添加特定属性
        if isinstance(item, Term):
            result["functor"] = item.functor
            result["arity"] = item.arity
            result["args"] = [self._process_item(arg) for arg in item.args]
            if item.probability is not None:
                result["probability"] = self._process_item(item.probability)
                
        elif isinstance(item, Var):
            result["name"] = item.name
            
        elif isinstance(item, Constant):
            result["value"] = item.value
            result["value_type"] = type(item.value).__name__
            
        elif isinstance(item, Clause):
            result["head"] = self._process_item(item.head)
            result["body"] = self._process_item(item.body)
            
        elif isinstance(item, AnnotatedDisjunction):
            result["heads"] = [self._process_item(head) for head in item.heads]
            result["body"] = self._process_item(item.body) if item.body else None
            
        elif isinstance(item, And):
            result["op1"] = self._process_item(item.op1)
            result["op2"] = self._process_item(item.op2)
            
        elif isinstance(item, Or):
            result["op1"] = self._process_item(item.op1)
            result["op2"] = self._process_item(item.op2)
            
        elif isinstance(item, Not):
            result["child"] = self._process_item(item.child)
            
        elif isinstance(item, AggTerm):
            result["functor"] = item.functor
            result["args"] = [self._process_item(arg) for arg in item.args]
            
        return result
    
    def pretty_print(self, parsed_result, indent=0):
        """
        美观打印解析结果
        
        :param parsed_result: 解析结果
        :param indent: 缩进级别
        """
        if isinstance(parsed_result, list):
            for item in parsed_result:
                self.pretty_print(item, indent)
            return
            
        indent_str = "  " * indent
        print(f"{indent_str}Type: {parsed_result['type']}")
        print(f"{indent_str}Content: {parsed_result['content']}")
        
        for key, value in parsed_result.items():
            if key not in ["type", "content", "raw"]:
                if isinstance(value, list):
                    print(f"{indent_str}{key}:")
                    for item in value:
                        self.pretty_print(item, indent+1)
                elif isinstance(value, dict):
                    print(f"{indent_str}{key}:")
                    self.pretty_print(value, indent+1)
                else:
                    print(f"{indent_str}{key}: {value}")
        print()

# 使用示例
def parse_problog(code_string):
    parser = ProblogParser()
    result = parser.parse(code_string)
    return result

# 打印示例
def print_parsed_problog(code_string):
    parser = ProblogParser()
    result = parser.parse(code_string)
    parser.pretty_print(result)
    return result

In [2]:
from problog.program import PrologString
from problog.logic import Term, Var, Constant, Clause, AnnotatedDisjunction, And, Or, Not, AggTerm
import inspect
class ImprovedProblogParser:
    """
    An optimized parser for ProbLog code that generates a cleaner, more efficient 
    structured representation with type annotations
    """
    
    def __init__(self):
        self.type_mapping = {
            Term: "term",
            Var: "variable",
            Constant: "constant",
            Clause: "clause",
            AnnotatedDisjunction: "annotated_disjunction",
            And: "conjunction",
            Or: "disjunction",
            Not: "negation",
            AggTerm: "aggregation"
        }
    
    def parse(self, code_string):
        """
        Parse ProbLog code and return a cleaner structured representation
        
        :param code_string: ProbLog code string
        :return: Structured representation with type annotations
        """
        program = PrologString(code_string)
        return [self._process_item(item) for item in program]
    
    def _process_item(self, item):
        """
        Process a single parsed item with improved representation
        
        :param item: Parsed item
        :return: Dictionary with type annotation and minimal necessary data
        """
        # Determine item type
        item_type = type(item)
        type_name = self.type_mapping.get(item_type, "unknown")
        
        # Create result with minimal common attributes
        result = {
            "type": type_name,
            "content": str(item)
        }
        
        # Add type-specific information without redundancy
        if isinstance(item, Var):
            result["name"] = item.name
            
        elif isinstance(item, Constant):
            result["value"] = item.value
            result["value_type"] = type(item.value).__name__
            
        elif isinstance(item, Term):
            # Only include these for regular Terms, not for specialized term types
            if type(item) == Term or isinstance(item, AggTerm):
                result["functor"] = item.functor
                result["arity"] = item.arity
                result["args"] = [self._process_item(arg) for arg in item.args]
                if item.probability is not None:
                    result["probability"] = self._process_item(item.probability)
                
        # Handle specialized term types with their specific properties
        if isinstance(item, Clause):
            result["head"] = self._process_item(item.head)
            result["body"] = self._process_item(item.body)
            
        elif isinstance(item, AnnotatedDisjunction):
            result["heads"] = [self._process_item(head) for head in item.heads]
            result["body"] = self._process_item(item.body) if item.body else None
            
        elif isinstance(item, And):
            result["op1"] = self._process_item(item.op1)
            result["op2"] = self._process_item(item.op2)
            
        elif isinstance(item, Or):
            result["op1"] = self._process_item(item.op1)
            result["op2"] = self._process_item(item.op2)
            
        elif isinstance(item, Not):
            result["child"] = self._process_item(item.child)
        
        # Special handling for lists to avoid deep nesting
        if isinstance(item, Term) and item.functor == "." and item.arity == 2:
            # Detect and handle Prolog lists specially
            elements, tail = self._extract_list_elements(item)
            result["type"] = "list"
            result["elements"] = [self._process_item(elem) for elem in elements]
            if tail.functor != "[]":  # Non-empty tail for non-standard lists
                result["tail"] = self._process_item(tail)
            
        return result
    
    def _extract_list_elements(self, list_term):
        """
        Extract elements from a Prolog list term
        
        :param list_term: Prolog list term
        :return: Tuple of (elements, tail)
        """
        elements = []
        current = list_term
        
        # Extract elements until we reach the end of the list or a non-standard tail
        while isinstance(current, Term) and current.functor == "." and current.arity == 2:
            elements.append(current.args[0])
            current = current.args[1]
        
        # Return elements and the tail (which is normally Term("[]"))
        return elements, current
        
    def pretty_print(self, parsed_result, indent=0):
        """
        Pretty print the parsed result
        
        :param parsed_result: Parsed result
        :param indent: Indentation level
        """
        if isinstance(parsed_result, list):
            for item in parsed_result:
                self.pretty_print(item, indent)
            return
            
        indent_str = "  " * indent
        print(f"{indent_str}Type: {parsed_result['type']}")
        print(f"{indent_str}Content: {parsed_result['content']}")
        
        for key, value in parsed_result.items():
            if key not in ["type", "content"]:
                if isinstance(value, list):
                    print(f"{indent_str}{key}:")
                    for item in value:
                        self.pretty_print(item, indent+1)
                elif isinstance(value, dict):
                    print(f"{indent_str}{key}:")
                    self.pretty_print(value, indent+1)
                else:
                    print(f"{indent_str}{key}: {value}")
        print()

# 使用示例
def parse_problog(code_string):
    parser = ImprovedProblogParser()
    result = parser.parse(code_string)
    return result

# 打印示例
def print_parsed_problog(code_string):
    parser = ImprovedProblogParser()
    result = parser.parse(code_string)
    parser.pretty_print(result)
    return result

In [ ]:
from problog.program import PrologString
from problog.logic import Term, Var, Constant, Clause, AnnotatedDisjunction, And, Or, Not, AggTerm, list2term, term2list, is_list

class ImprovedProblogParser:
    """
    An optimized parser for ProbLog code that generates a cleaner, more efficient 
    structured representation with type annotations
    """
    
    def __init__(self):
        self.type_mapping = {
            Term: "term",
            Var: "variable",
            Constant: "constant",
            Clause: "clause",
            AnnotatedDisjunction: "annotated_disjunction",
            And: "conjunction",
            Or: "disjunction",
            Not: "negation",
            AggTerm: "aggregation"
        }
    
    def parse(self, code_string):
        """
        Parse ProbLog code and return a cleaner structured representation
        args:
            code_string: ProbLog code string
        return: Structured representation with type annotations
        """
        program = PrologString(code_string)
        return [self._process_item(item) for item in program]
    
    def _process_item(self, item):
        """
        Process a single parsed item with improved representation
        args:   
            item: Parsed item
        return: Dictionary with type annotation and minimal necessary data
        """
        # Determine item type
        item_type = type(item)
        type_name = self.type_mapping.get(item_type, "unknown")
        
        # Create result with minimal common attributes
        result = {
            "_type": type_name,
            "_content": str(item)
        }

        # Add type-specific information without redundancy
        if isinstance(item, Var):
            result["name"] = item.name
            
        elif isinstance(item, Constant):
            result["value"] = item.value
            
        elif isinstance(item, Term):
            result["_functor"] = item.functor
            # Handle lists specially - keep them as a single cohesive structure
            if is_list(item):
                # Extract list elements and process them
                elements, tail = self._extract_list_elements(item)
                result["_type"] = "list"
                result["args"] = [self._process_item(elem) for elem in elements]
                # Only include tail if it's not the empty list
                if not (isinstance(tail, Term) and tail.functor == "[]"):
                    result["_tail"] = self._process_item(tail)

            # Handle regular terms
            else:
                result["_arity"] = item.arity
                result["args"] = [self._process_item(arg) for arg in item.args]
                if item.probability is not None:
                    result["_probability"] = self._process_item(item.probability)

        # Handle specialized term types with their specific properties
        if isinstance(item, Clause):
            result["_head"] = self._process_item(item.head)
            result["body"] = self._process_item(item.body)

        elif isinstance(item, AnnotatedDisjunction):
            result["_heads"] = [self._process_item(head) for head in item.heads]
            result["body"] = self._process_item(item.body) if item.body else None

        elif isinstance(item, And):
            result["_functor"] = "And"
            result["AND1"] = self._process_item(item.op1)
            result["AND2"] = self._process_item(item.op2)
            
        elif isinstance(item, Or):
            result["_functor"] = "Or"
            result["OR1"] = self._process_item(item.op1)
            result["OR2"] = self._process_item(item.op2)

        elif isinstance(item, Not):
            result["_functor"] = "Not"
            result["NOT"] = self._process_item(item.child)

        return result
    
    def _extract_list_elements(self, list_term):
        """
        Extract elements from a Prolog list term
        args:
            list_term: Prolog list term
        return: Tuple of (elements, tail)
        """
        elements = []
        current = list_term
        
        # Extract elements until we reach the end of the list or a non-standard tail
        while isinstance(current, Term) and current.functor == "." and current.arity == 2:
            elements.append(current.args[0])
            current = current.args[1]
        
        # Return elements and the tail (which is normally Term("[]"))
        return elements, current


def parse_problog(code_string):
    parser = ImprovedProblogParser()
    result = parser.parse(code_string)
    return result

In [ ]:
from problog.program import PrologString
from problog.logic import Term, Var, Constant, Clause, AnnotatedDisjunction, And, Or, Not, AggTerm, list2term, term2list, is_list
import re

class ImprovedProblogParser:
    """
    An optimized parser for ProbLog code that generates a cleaner, more efficient 
    structured representation with type annotations
    """
    
    def __init__(self):
        self.type_mapping = {
            Term: "term",
            Var: "variable",
            Constant: "constant",
            Clause: "clause",
            AnnotatedDisjunction: "annotated_disjunction",
            And: "conjunction",
            Or: "disjunction",
            Not: "negation",
            AggTerm: "aggregation"
        }
    
    def parse(self, code_string):
        """
        Parse ProbLog code and return a cleaner structured representation
        args:
            code_string: ProbLog code string
        return: Structured representation with type annotations
        """
        # Extract comments before parsing
        lines = code_string.split('\n')
        code_without_comments = []
        comments_map = {}  # Maps line number to comments
        
        for i, line in enumerate(lines):
            # Extract single-line comments
            single_comment_match = re.search(r'%(.*)$', line)
            if single_comment_match:
                comments_map[i] = single_comment_match.group(1).strip()
                # Remove comment from line for parsing
                line = re.sub(r'%.*$', '', line)
            
            # Only add non-empty lines
            if line.strip():
                code_without_comments.append(line)
        
        # Parse the code without comments
        clean_code = '\n'.join(code_without_comments)
        program = PrologString(clean_code)
        
        # Process each item and try to associate with comments
        results = []
        current_line = 0
        
        for item in program:
            processed_item = self._process_item(item)
            
            # Try to find associated comments
            item_str = str(item)
            
            # Find which line this item starts on by comparing with original code
            for i, orig_line in enumerate(lines):
                if current_line <= i:
                    # Check if this item appears in this line
                    if any(part.strip() in orig_line for part in item_str.split() if part.strip()):
                        # Add inline comment if exists
                        if i in comments_map:
                            processed_item["_comment"] = comments_map[i]
                        current_line = i + 1
                        break
            
            results.append(processed_item)
        
        return results
    
    def _process_item(self, item):
        """
        Process a single parsed item with improved representation
        args:   
            item: Parsed item
        return: Dictionary with type annotation and minimal necessary data
        """
        # Determine item type
        item_type = type(item)
        type_name = self.type_mapping.get(item_type, "unknown")
        
        # Create result with minimal common attributes
        result = {
            "_type": type_name,
            "_content": str(item)
        }

        # Add type-specific information without redundancy
        if isinstance(item, Var):
            result["name"] = item.name
            
        elif isinstance(item, Constant):
            result["value"] = item.value
            
        elif isinstance(item, Term):
            result["_functor"] = item.functor
            # Handle lists specially - keep them as a single cohesive structure
            if is_list(item):
                # Extract list elements and process them
                elements, tail = self._extract_list_elements(item)
                result["_type"] = "list"
                result["args"] = [self._process_item(elem) for elem in elements]
                # Only include tail if it's not the empty list
                if not (isinstance(tail, Term) and tail.functor == "[]"):
                    result["_tail"] = self._process_item(tail)

            # Handle regular terms
            else:
                result["_arity"] = item.arity
                result["args"] = [self._process_item(arg) for arg in item.args]
                if item.probability is not None:
                    result["_probability"] = self._process_item(item.probability)

        # Handle specialized term types with their specific properties
        if isinstance(item, Clause):
            result["_head"] = self._process_item(item.head)
            result["body"] = self._process_item(item.body)

        elif isinstance(item, AnnotatedDisjunction):
            result["_heads"] = [self._process_item(head) for head in item.heads]
            result["body"] = self._process_item(item.body) if item.body else None

        elif isinstance(item, And):
            result["_functor"] = "And"
            result["AND1"] = self._process_item(item.op1)
            result["AND2"] = self._process_item(item.op2)
            
        elif isinstance(item, Or):
            result["_functor"] = "Or"
            result["OR1"] = self._process_item(item.op1)
            result["OR2"] = self._process_item(item.op2)

        elif isinstance(item, Not):
            result["_functor"] = "Not"
            result["NOT"] = self._process_item(item.child)

        return result
    
    def _extract_list_elements(self, list_term):
        """
        Extract elements from a Prolog list term
        args:
            list_term: Prolog list term
        return: Tuple of (elements, tail)
        """
        elements = []
        current = list_term
        
        # Extract elements until we reach the end of the list or a non-standard tail
        while isinstance(current, Term) and current.functor == "." and current.arity == 2:
            elements.append(current.args[0])
            current = current.args[1]
        
        # Return elements and the tail (which is normally Term("[]"))
        return elements, current


def parse_problog(code_string):
    parser = ImprovedProblogParser()
    result = parser.parse(code_string)
    return result

# 为了更好地处理多行注释，我们可以添加一个专门的注释处理器
def handle_multiline_comments(code_string):
    """
    Handle multiline comments /* ... */
    """
    pattern = r'/\*[\s\S]*?\*/'
    comments = []
    
    def replace_comment(match):
        comment_text = match.group(0)
        comments.append(comment_text)
        # Replace with a placeholder that includes line breaks
        return '\n' * comment_text.count('\n')
    
    cleaned_code = re.sub(pattern, replace_comment, code_string)
    return cleaned_code, comments

# 更完整的注释处理版本
def parse_problog_with_all_comments(code_string):
    """
    Parse ProbLog code while preserving all types of comments
    """
    # First handle multiline comments
    cleaned_code, multiline_comments = handle_multiline_comments(code_string)
    
    parser = ImprovedProblogParser()
    result = parser.parse(cleaned_code)
    
    # Add multiline comments to the result if needed
    if multiline_comments:
        result.append({
            "_type": "multiline_comments",
            "_content": multiline_comments
        })
    
    return result

In [3]:
from problog.program import PrologString
from problog.logic import Term, Var, Constant, Clause, AnnotatedDisjunction, And, Or, Not, AggTerm, list2term, term2list, is_list
import re

class ImprovedProblogParser:
    """
    An optimized parser for ProbLog code that generates a cleaner, more efficient 
    structured representation with type annotations
    """
    
    def __init__(self):
        self.type_mapping = {
            Term: "term",
            Var: "variable",
            Constant: "constant",
            Clause: "clause",
            AnnotatedDisjunction: "annotated_disjunction",
            And: "conjunction",
            Or: "disjunction",
            Not: "negation",
            AggTerm: "aggregation"
        }
    
    def parse(self, code_string):
        """
        Parse ProbLog code and return a cleaner structured representation
        args:
            code_string: ProbLog code string
        return: Structured representation with type annotations
        """
        # Extract comments before parsing
        lines = code_string.split('\n')
        code_without_comments = []
        comments_map = {}  # Maps line number to comments
        
        for i, line in enumerate(lines):
            # Extract single-line comments
            single_comment_match = re.search(r'%(.*)$', line)
            if single_comment_match:
                comments_map[i] = single_comment_match.group(1).strip()
                # Remove comment from line for parsing
                line = re.sub(r'%.*$', '', line)
            
            # Only add non-empty lines
            if line.strip():
                code_without_comments.append(line)
        
        # Parse the code without comments
        clean_code = '\n'.join(code_without_comments)
        program = PrologString(clean_code)
        
        # Process each item and try to associate with comments
        results = []
        current_line = 0
        
        for item in program:
            processed_item = self._process_item(item)
            
            # Try to find associated comments
            item_str = str(item)
            
            # Find which line this item starts on by comparing with original code
            for i, orig_line in enumerate(lines):
                if current_line <= i:
                    # Check if this item appears in this line
                    if any(part.strip() in orig_line for part in item_str.split() if part.strip()):
                        # Add inline comment if exists
                        if i in comments_map:
                            processed_item["_comment"] = comments_map[i]
                        current_line = i + 1
                        break
            
            results.append(processed_item)
        
        return results
    
    def _process_item(self, item):
        """
        Process a single parsed item with improved representation
        args:   
            item: Parsed item
        return: Dictionary with type annotation and minimal necessary data
        """
        # Determine item type
        item_type = type(item)
        type_name = self.type_mapping.get(item_type, "unknown")
        
        # Create result with minimal common attributes
        result = {
            "_type": type_name,
            "_content": str(item)
        }

        # Add type-specific information without redundancy
        if isinstance(item, Var):
            result["name"] = item.name
            
        elif isinstance(item, Constant):
            result["value"] = item.value
            
        elif isinstance(item, Term):
            result["_functor"] = item.functor
            # Handle lists specially - keep them as a single cohesive structure
            if is_list(item):
                # Extract list elements and process them
                elements, tail = self._extract_list_elements(item)
                result["_type"] = "list"
                result["args"] = [self._process_item(elem) for elem in elements]
                # Only include tail if it's not the empty list
                if not (isinstance(tail, Term) and tail.functor == "[]"):
                    result["_tail"] = self._process_item(tail)

            # Handle regular terms
            else:
                result["_arity"] = item.arity
                result["args"] = [self._process_item(arg) for arg in item.args]
                if item.probability is not None:
                    result["_probability"] = self._process_item(item.probability)

        # Handle specialized term types with their specific properties
        if isinstance(item, Clause):
            result["_head"] = self._process_item(item.head)
            result["body"] = self._process_item(item.body)

        elif isinstance(item, AnnotatedDisjunction):
            result["_heads"] = [self._process_item(head) for head in item.heads]
            result["body"] = self._process_item(item.body) if item.body else None

        elif isinstance(item, And):
            result["_functor"] = "And"
            result["AND1"] = self._process_item(item.op1)
            result["AND2"] = self._process_item(item.op2)
            
        elif isinstance(item, Or):
            result["_functor"] = "Or"
            result["OR1"] = self._process_item(item.op1)
            result["OR2"] = self._process_item(item.op2)

        elif isinstance(item, Not):
            result["_functor"] = "Not"
            result["NOT"] = self._process_item(item.child)

        return result
    
    def _extract_list_elements(self, list_term):
        """
        Extract elements from a Prolog list term
        args:
            list_term: Prolog list term
        return: Tuple of (elements, tail)
        """
        elements = []
        current = list_term
        
        # Extract elements until we reach the end of the list or a non-standard tail
        while isinstance(current, Term) and current.functor == "." and current.arity == 2:
            elements.append(current.args[0])
            current = current.args[1]
        
        # Return elements and the tail (which is normally Term("[]"))
        return elements, current


def parse_problog(code_string):
    parser = ImprovedProblogParser()
    result = parser.parse(code_string)
    return result

# 为了更好地处理多行注释，我们可以添加一个专门的注释处理器
def handle_multiline_comments(code_string):
    """
    Handle multiline comments /* ... */
    """
    pattern = r'/\*[\s\S]*?\*/'
    comments = []
    
    def replace_comment(match):
        comment_text = match.group(0)
        comments.append(comment_text)
        # Replace with a placeholder that includes line breaks
        return '\n' * comment_text.count('\n')
    
    cleaned_code = re.sub(pattern, replace_comment, code_string)
    return cleaned_code, comments

# 更完整的注释处理版本
def parse_problog_with_all_comments(code_string):
    """
    Parse ProbLog code while preserving all types of comments
    """
    # First handle multiline comments
    cleaned_code, multiline_comments = handle_multiline_comments(code_string)
    
    parser = ImprovedProblogParser()
    result = parser.parse(cleaned_code)
    
    # Add multiline comments to the result if needed
    if multiline_comments:
        result.append({
            "_type": "multiline_comments",
            "_content": multiline_comments
        })
    
    return result

In [4]:
model = """
% UAV properties
initial_charge ~ normal(90, 5).
charge_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

% Sufficient charge to return to operator
can_return(X) :-
    B is initial_charge, O is charge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D).

sensitive_section(X) :-
    langda(LLM:"/* Secure */",FUP:"True").

security(X) :-
    vlos(X),
    sensitive_section(X).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

% Definition of a valid mission
landscape(X) :- 
    security(X), weight < 25, can_return(X); 
    permits(X), can_return(X).
"""

addition = """
lann(mnist_net:"mnist network",[X],Y,[0,1,2,3,4,5,6,7,8,9]:"pick the output from one of there value") :: digit(X,Y).
% Hi there first commend
number([],Result,Result).% Hi there second commend
number([H|T],Acc,Result) :- langda(NET:"use mnist network",LLM:"Recursively process the image list: identify the digits one by one and construct the corresponding multi-bit integers in low-bit first order.").
number(X,Y) :- number(X,0,Y). % Hi third first commend /* Multiline comment!  Multiline comment! Multiline comment!
 Multiline comment! Multiline comment!
*/

multi_addition(X,Y,Z) :- langda(NET:"use mnist network",LLM:"Convert two sets of sequences consisting of digital pictures into integers and add them together to get the numerical result.",FUP:"False"). % Hi there 4th commend

addition(X,Y,Z) :- langda(NET:"use mnist network",LLM:"Use a neural network to identify the numbers in two pictures and then calculate their sum.").
"""
full_text_for_parse_test = '''
% =============================== % EXAMPLE % =============================== %
lann(mnist_net1:"decrib net1",[M]:"describe M",N,[0,1,"None"]:"desc someting") :: event1(M,N). % lann(what?)
lann(mnist_net2,[X],Y:"what is Y?",[2,3,"dog","horn"]:"output 2 and 3")    % (what_is_this(X,S):maybe?
:: funfunf(X,Y).     % have fun here)!

lann(mnist_net3,       % net work haha
[R],S:"what whatwhaitsat?",     % X and what Y: this Y! stop, lann(X,Y)
["this","thas","was","isss"]:"outoutoutout")    % (what_e q2wre g wads  cf :: event114(X,Y).
:: digit232(R,S).     % have fun here)!

% UAV properties
initial_langda_content ~ normal(90, 5).
langda_contente_cost ~ normal(-0.1, 0.2).
weight ~ normal(0.2, 0.1).

% Weather conditions
1/10::fog; 9/10::clear.

% Visual line of sight
vlos(X) :- 
    fog, distance(X, operator) < 50;
    clear, distance(X, operator) < 100;
    clear, over(X, bay), distance(X, operator) < 400.

langda(ans:"answer",LLM:"nothing"). % langda(LLM:"this is me").
% langda(X:"answer",LLM). % stop it
langda(why:"answer",NET:"[mlp(1,2,3,4,5,6)]").
langda(X:"answer",llm:"stop"). % why not here?
/*
% Sufficient langda_content[i]ge to return to operator
can_return(X) :- 
    B is initial_langda_content[i]ge, O is langda_content[i]ge_cost,
    D is distance(X, operator), 0 < B + (2 * O * D), langda(LLM:"how about this?")
*/can_return(X) :- 
    B is initial_langda_content, O is langda_content,
    D is distance(X, operator), 0 < B + (2 * O * D), langda(LLM:"how about this?").

langda(NET:"[no(1,2,3)]",LLM:"This is crazy")        .       waht_coin(sdaw,A) :- zzz(X,Y),
langda(LLM,NET).

% Permits related to local features
permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).

valid_drone_flight(X) :- (langda(X:"Return",T,LLM:"there's a high building at: (100,200)",NET:"[mnist_net1(0,1), mnist_net2(2,3)]"), vlos(X));(can_return(X));langda(T:"anstimewer",NET).

coin_win() :-
    % the code need to be clean!
    langda(X:"return", T:"time", NET:"[mnist_net1(0,1), mnist_net2(2,3)]",              % a langda code for new stuff
    Y:"happy",              % why the code is happy?
    TOL:"[mnist_net1(0,1), mnist_net2(2,3)]",              % wewewewewewewe
    LLM:"If the code difference is 2, % it returns 1, otherwise it returns 0.")          % we use deepseek here!
    ,permits(X), langda(X:"return", Y:"fun", Z:"darmstadt", NET:"[position_net(there,here), audio(dog,child,horn)]",     % nothing useful % so quit!
    LLM:"If the we is 2,            % comment end
    and horn is there, we eat banana.
    permits(X) :- 
    distance(X, service) < 15; distance(X, primary) < 15;
    distance(X, secondary) < 10; distance(X, tertiary) < 5;
    distance(X, crossing) < 5; distance(X, rail) < 5;
    over(X, park).
    "). coin_this(X,Y,3) :- langda(X,LLM:"asdasda"),       % comment conisthis

    predicate(sd, X), asd(MMMA)
    . 

% Definition of a valid mission
landscape(X) :- 
    vlos(X), weight < 25, can_return(X); 
    permits(X), can_return(X).

'''

annotated_dis = "rain(0.3); sunny(0.7) :- cloudy."
list_test = """lann(mnist_net1:"decrib net1",[M]:"describe M",N,[0,1,"None"]:"desc someting") :: event1(M,N). % lann(what?)"""
import pprint
from config import paths

# result4print = print_parsed_problog(addition)
# pprint.pp(result4print)

result = parse_problog_with_all_comments(model)
# result = parse_problog(full_text_for_parse_test)
ppresult = pprint.pformat(result)
print(ppresult)
paths.save_as_file(ppresult,"result","parser_test")

[{'_arity': 2,
  '_content': 'initial_charge::normal(90,5)',
  '_functor': 'normal',
  '_probability': {'_arity': 0,
                   '_content': 'initial_charge',
                   '_functor': 'initial_charge',
                   '_type': 'term',
                   'args': []},
  '_type': 'term',
  'args': [{'_content': '90', '_type': 'constant', 'value': 90},
           {'_content': '5', '_type': 'constant', 'value': 5}]},
 {'_arity': 2,
  '_content': 'charge_cost::normal(-0.1,0.2)',
  '_functor': 'normal',
  '_probability': {'_arity': 0,
                   '_content': 'charge_cost',
                   '_functor': 'charge_cost',
                   '_type': 'term',
                   'args': []},
  '_type': 'term',
  'args': [{'_content': '-0.1', '_type': 'constant', 'value': -0.1},
           {'_content': '0.2', '_type': 'constant', 'value': 0.2}]},
 {'_arity': 2,
  '_content': 'weight::normal(0.2,0.1)',
  '_functor': 'normal',
  '_probability': {'_arity': 0,
                   '_

PosixPath('/Users/zhenzhili/MASTERTHESIS/#Expert_System_Design/examples/LANGDA/#drone_with_dpl_in_langgraph_react_v6.2/history/parser_test_result.txt')